In [1]:
import numpy as np

In [2]:
import pandas as pd


In [3]:
from mlxtend.frequent_patterns import apriori, association_rules 


In [4]:
url = "https://github.com/NiteshTuladhar/Data-Warehouse-data-sets/blob/master/Online%20Retail.csv"

In [9]:
data = pd.read_excel('C://Users/user/Data warehouse/Online_rRetail.xls')

In [10]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [12]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [13]:
data.Country.unique() 

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain'], dtype=object)

In [14]:
data['Description'] = data['Description'].str.strip() 

In [15]:
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
data['InvoiceNo'] = data['InvoiceNo'].astype('str') 

In [16]:
data = data[~data['InvoiceNo'].str.contains('C')] 

Splitting the data according to the region of transaction

In [17]:
basket_France = (data[data['Country'] =="France"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [18]:
basket_UK = (data[data['Country'] =="United Kingdom"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo'))

In [19]:
basket_Por = (data[data['Country'] =="Portugal"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [20]:
basket_Sweden = (data[data['Country'] =="Sweden"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

Hot encoding the Data

In [21]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1

In [22]:
basket_encoded = basket_France.applymap(hot_encode) 
basket_France = basket_encoded 
  
basket_encoded = basket_UK.applymap(hot_encode) 
basket_UK = basket_encoded 
  
basket_encoded = basket_Por.applymap(hot_encode) 
basket_Por = basket_encoded 
  
basket_encoded = basket_Sweden.applymap(hot_encode) 
basket_Sweden = basket_encoded 

Step 6: Buliding the models and analyzing the results

In [23]:
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True) 

In [24]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

                          antecedents                   consequents  \
156      (OFFICE MUG WARMER POLKADOT)    (BLACK RECORD COVER FRAME)   
157        (BLACK RECORD COVER FRAME)  (OFFICE MUG WARMER POLKADOT)   
292              (SET/4 SKULL BADGES)   (DOORMAT RESPECTABLE HOUSE)   
293       (DOORMAT RESPECTABLE HOUSE)          (SET/4 SKULL BADGES)   
518  (SKULLS  WATER TRANSFER TATTOOS)      (OFFICE MUG WARMER PINK)   

     antecedent support  consequent support   support  confidence  lift  \
156            0.071429            0.071429  0.071429         1.0  14.0   
157            0.071429            0.071429  0.071429         1.0  14.0   
292            0.071429            0.071429  0.071429         1.0  14.0   
293            0.071429            0.071429  0.071429         1.0  14.0   
518            0.071429            0.071429  0.071429         1.0  14.0   

     leverage  conviction  
156  0.066327         inf  
157  0.066327         inf  
292  0.066327         inf  
293  0.066